# imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import csv
#import pyedflib
import heartbeat as hb
import preparation_hypno as preph
import preparation_edf as prepedf
import neurokit2 as nk
import matplotlib.pyplot as plt
from edfrd import read_header, read_data_records
from datetime import time
from datetime import datetime
from datetime import date
from datetime import timedelta
import edfrd
from neurokit2.hrv.hrv_utils import _hrv_get_rri
#from pyedflib.highlevel import anonymize_edf, read_edf

# allocate variables

In [51]:
#öffnen des Hypnogram
filename = 'Probanden/NihonKhoden/007/150921EpochReport.txt'
def fileopener(filename):
    return pd.read_csv(filename, skiprows=0, delimiter="\t", sep="\t")
hypno = fileopener(filename)
hypno['Stg']

0       L
1       L
2       L
3       L
4       L
       ..
944    N2
945    N2
946    N2
947     W
948     W
Name: Stg, Length: 949, dtype: object

In [27]:
file_path = 'Probanden/NihonKhoden/007/15092021.edf'

In [31]:
#öffnen der EDF
header = read_header(file_path)
header

Header(version='0', local_patient_identification='GA067096 M X GA067096_GA067096', local_recording_identification='GA067096', startdate_of_recording='15.09.21', starttime_of_recording='22.31.14', number_of_bytes_in_header_record=5120, reserved=None, number_of_data_records=26334, duration_of_a_data_record=1, number_of_signals=19, signals=(SignalHeader(label='EOG Li', transducer_type='Unknown', physical_dimension='uV', physical_minimum=-3200.0, physical_maximum=3200.0, digital_minimum=-32768, digital_maximum=32767, prefiltering='HP:0,300Hz LP:35Hz N:50Hz', nr_of_samples_in_each_data_record=500, reserved=None), SignalHeader(label='EOG Re', transducer_type='Unknown', physical_dimension='uV', physical_minimum=-3200.0, physical_maximum=3200.0, digital_minimum=-32768, digital_maximum=32767, prefiltering='HP:0,300Hz LP:35Hz N:50Hz', nr_of_samples_in_each_data_record=500, reserved=None), SignalHeader(label='F3-A2', transducer_type='Unknown', physical_dimension='uV', physical_minimum=-3200.0, ph

In [19]:
header.signals[9]

SignalHeader(label='EKG', transducer_type='Unknown', physical_dimension='uV', physical_minimum=-3200.0, physical_maximum=3200.0, digital_minimum=-32768, digital_maximum=32767, prefiltering='HP:0,300Hz LP:100Hz N:50Hz', nr_of_samples_in_each_data_record=500, reserved=None)

In [24]:
ecg_channel = prepedf.get_ecg_channel(file_path)

Channel : 6


# Calculation cutpoints from seperate hypnogram

In [ ]:
#cutpoints where stadium changes

In [ ]:
cutpoints = preph.calculate_cutpoints(dfh)
cutpoints

In [ ]:
diff = preph.calculate_cutpoints_diff(cutpoints)
print(diff)

# Calculation difference start from seperate hypnogram and edf

In [ ]:
startedf = header.starttime_of_recording
starthypno = dfh.iloc[0][0]
print(starthypno)

# DTF
startedfdto = datetime.strptime(startedf, "%H.%M.%S")

# hypno
starthypnodto = datetime.strptime(starthypno, "%H:%M:%S")

difference_between = prepedf.get_absolute_time_diff(startedfdto, starthypnodto)
print(difference_between)

# cut first diff with difference, diff acts as a pattern for cutting

In [ ]:
diff[0] = int(diff[0]-difference_between)
print(diff)

In [ ]:
def stuecke(diff):
    zaehler = 0
    for i in diff:
        bla = i/300
        print(bla)
        zaehler = zaehler + int(bla)
        print(zaehler)
    return zaehler    

stuecke(diff)

In [ ]:
#get ecg from EDF as stadium sections

In [ ]:
ecg_signal = prepedf.get_data_sections(file_path,ecg_channel,diff,True)

In [ ]:
#check if length are correct, ecg has to be one longer than diff

In [ ]:
print(len(diff))
print(len(ecg_signal))

In [ ]:
print(diff[0])
print(len(ecg_signal[0])/256)
len(ecg_signal[0])

In [ ]:
#example plotting for first 3 seconds

In [ ]:
plt.rcParams['figure.figsize'] = [15, 9] 
plt.plot(ecg_signal[0], alpha=0.5, color='blue', label="raw signal")
plt.xlim(0,768)
plt.show()

In [ ]:
#Liste anlegen aus Epochen und Stadienbezeichnung
list_of_stad_value = []

for i,value in enumerate(ecg_signal):
    list_of_stad_value.append({'stadium':cutpoints['Stadium'][i], 'values': value})

#print(list_of_stad_value[62]['stadium'])

In [ ]:
print(len(list_of_stad_value))

In [ ]:
segments = prepedf.segment_epochs(list_of_stad_value)
len(segments[1]['values'])

In [ ]:
prepedf.plot_compare_signals(segments[1]['values'], 256)

In [ ]:
test = segments[1]['values']
peaktest = test[42000:47000]
#for i in test:
 #   if i>70:
  #      test.pop()
len(test)
#sig = nk.signal_simulate(duration=20, sampling_rate=256, frequency=1)
prepedf.compare_peaks(test)

In [ ]:
#results
hrv_results,hrv_process,hrv_info = prepedf.analyse_segments(idp, 256, segments)
#testplot(segment)
#test_analyze(segment)

In [ ]:
hrv_results